In [82]:
# import necessary python modules
import csv
import pandas as pd
import numpy as np

# Create new data frames and import raw data
---

Downloaded Excel files were re-saved as CSV. The following lines import those files for each state and create our four primary tables with the raw data from BLM.

In [83]:
# create a list of the states we'll be analyzing, will be used frequently
statelist = ["AZ", "CA", "CO", "ID", "MT", "NM", "NV", "OR", "UT", "WY"]

In [84]:
# Create a new empty DataFrame for each table
allotments = pd.DataFrame()
operators = pd.DataFrame()
permits = pd.DataFrame()
health = pd.DataFrame()

#create a variable storing the path for each set of raw files
allotments_path = 'data/rangeland-administration-system/allotment-info/'
operators_path = 'data/rangeland-administration-system/operator-info/'
permits_path = 'data/rangeland-administration-system/permit-schedule-info/'
health_path = 'data/rangeland-health/'

# Import those files for allotments ... (probably should be a function)
for s in statelist:
    csv_file = '{}{}.csv'.format(allotments_path, s)
    new_data = pd.read_csv(csv_file, dtype={'Allotment Number': 'object', 'Auth No': 'object'})
    allotments = allotments.append(new_data)

# ... and for operators
for s in statelist:
    csv_file = '{}{}.csv'.format(operators_path, s)
    new_data = pd.read_csv(csv_file, dtype={'Allotment Number': 'object', 'Auth No': 'object'})
    operators = operators.append(new_data)

# ... and for permits
for s in statelist:
    csv_file = '{}{}.csv'.format(permits_path, s)
    new_data = pd.read_csv(csv_file, dtype={'Allotment Number': 'object', 'Auth No': 'object'})
    permits = permits.append(new_data)

# ... and for land health standards
for s in statelist:
    csv_file = '{}{}.csv'.format(health_path, s)
    new_data = pd.read_csv(csv_file, dtype={'Allotment Number': 'object', 'Auth No': 'object'})
    health = health.append(new_data)

# Processing for Django

###  Field Offices table
---

The following lines of code creates a table of field offices by dropping duplicates and adding a unique ID as an integer. This ID can be referenced by other tables in Django.


In [85]:
# create a new df with field office info
field_offices = allotments[["Admin Office", "Field Office"]]

In [86]:
#  select only uniques from the DB
field_offices.drop_duplicates('Admin Office', inplace = True)

In [87]:
# save and re-read as a CSV for a janky but fast way of generating unique IDs starting at 1
field_offices.to_csv('data/processed/field_offices.csv')
field_offices = pd.read_csv('data/processed/field_offices.csv')

In [88]:
# Create a new column for field office state.
field_offices["State"] = field_offices["Admin Office"]
field_offices["State"] = field_offices["State"].str[2:4]

In [89]:
field_offices.loc[field_offices['State'] == 'AZ', 'StateCode'] = 1
field_offices.loc[field_offices['State'] == 'CA', 'StateCode'] = 2
field_offices.loc[field_offices['State'] == 'CO', 'StateCode'] = 3
field_offices.loc[field_offices['State'] == 'ID', 'StateCode'] = 4
field_offices.loc[field_offices['State'] == 'MT', 'StateCode'] = 5
field_offices.loc[field_offices['State'] == 'NM', 'StateCode'] = 6
field_offices.loc[field_offices['State'] == 'NV', 'StateCode'] = 7
field_offices.loc[field_offices['State'] == 'OR', 'StateCode'] = 8
field_offices.loc[field_offices['State'] == 'UT', 'StateCode'] = 9
field_offices.loc[field_offices['State'] == 'WY', 'StateCode'] = 10

field_offices['StateCode'] = field_offices['StateCode'].astype('int64')

In [90]:
field_offices['id'] = field_offices.index
field_offices['id'] = field_offices['id'] + 1

In [91]:
field_offices=field_offices.rename(columns = {'Admin Office':'office_code', 'Field Office': 'office_name', 'StateCode': 'state_id'})

In [92]:
field_offices = field_offices[['id', 'office_code', 'office_name', 'state_id']]
field_offices.to_csv("data/processed/field_offices.csv")

###Operators table
---

The following lines of code create a table of operators (ranches) with a unique ID that can be referenced by other tables, such as the permits table. Create a table of operators linked to field offices, and with a unique ID that can be referencecd by other tables

In [93]:
#rename operators columsn to remove spaces and capital
operators=operators.rename(columns = {'Off CD': 'office_code', 'Auth No':'auth_no', 'Operator Display Name': 'operator_display_name', 'Address1': 'address1', 'Address2': 'address2', 'City': 'city', 'Phone Number': 'phone_number', 'Release Text': 'release_text', 'Zipcode1 5': 'zipcode15', 'Zipcode6 9': 'zipcode69'})

In [94]:
#assign an id based on the index, but skip the 0
operators["id"] = operators.index + 1

In [95]:
#create a new dataframe that joins operators with field offices
new_ops = pd.merge(operators, field_offices, on='office_code', how='inner')

In [96]:
#that worked great now overwrite operators with that same data
operators = new_ops

In [97]:
#rename the two different id fields so we've got what we want.
operators['id'] = operators['id_x'].astype(int)
operators['field_office_id'] = operators['id_y'].astype(int)

In [98]:
# shed the data we don't want by reassigning the variable name 'operators' new a new dataframe with only these columns selected
operators = operators[['id', 'auth_no', 'operator_display_name', 'address1', 'address2', 'city', 'zipcode15', 'zipcode69', 'ST2', 'phone_number', 'release_text', 'field_office_id']]

In [99]:
# concatenate the ranch name and the city to create a unique identifier, just in case
# operator names appear to have no duplicates
operators['unique'] = operators['operator_display_name'] + operators['city']

/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [100]:
# check the first 10 rows
operators['unique'][:10]

0    NOEL,MICHAEL E,SHERRY M;STEWART,TREVOR D,JENNI...
1              DUANE E. AND IVONA H. MAGOONBUNKERVILLE
2                 LARSON, L. LYNN AND SANDRAST. GEORGE
3                     SUNSHINE CATTLE CO. LCST. GEORGE
4                       HEATON CATTLE COMPANYST GEORGE
5                            RUESCH, DEVIN D,HURRICANE
6                         DIAMOND D RANCH INCST GEORGE
7                        NIELSON, WILLIAM M.ST. GEORGE
8                       PARIA VALLEY LIVESTOCK LLCPAGE
9                           DARRELL E. BLAKEST. GEORGE
Name: unique, dtype: object

In [101]:
# create a new datafraem and drop any duplicates, checking for duplicates using our new unique field
# operators_unique = operators
# operators_unique.drop_duplicates('unique', inplace=True)

In [102]:
#create a new dataframe and fill it with operators data
unique_ops = pd.DataFrame()
unique_ops = unique_ops.append(operators)

In [103]:
#drop the duplciates out of it 
unique_ops.drop_duplicates('unique', inplace=True)

In [104]:
#assign a numeric 'id' field
unique_ops['operator_id'] = unique_ops.index + 1

In [105]:
#and write the necessary fields into a csv
unique_ops[['operator_display_name', 'address1', 'address2', 'city', 'zipcode15', 'zipcode69', 'phone_number', 'release_text', 'operator_id']].to_csv('data/processed/operators.csv')

In [106]:
unique_ops.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15327 entries, 0 to 17862
Data columns (total 14 columns):
id                       15327 non-null int64
auth_no                  15327 non-null object
operator_display_name    15327 non-null object
address1                 15327 non-null object
address2                 504 non-null object
city                     15327 non-null object
zipcode15                15323 non-null object
zipcode69                1383 non-null object
ST2                      15327 non-null object
phone_number             3952 non-null object
release_text             15327 non-null object
field_office_id          15327 non-null int64
unique                   15327 non-null object
operator_id              15327 non-null int64
dtypes: int64(3), object(11)
memory usage: 1.8+ MB


### Authorization numbers table
---
Create a table that links authorization numbers to operators.

In [107]:
authorizations = operators[['operator_display_name', 'address1', 'auth_no']]

In [108]:
authorizations['unique'] = authorizations['operator_display_name'] + authorizations['address1']
authorizations['id'] = authorizations.index + 1

/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from IPython.kernel.zmq import kernelapp as app


In [109]:
# authorizations

In [110]:
#check the first fiew rows of the file
authorizations[:3]

,operator_display_name,address1,auth_no,unique,id
0,"NOEL,MICHAEL E,SHERRY M;STEWART,TREVOR D,JENNI...",P O BOX 301,0200054,"NOEL,MICHAEL E,SHERRY M;STEWART,TREVOR D,JENNI...",1
1,DUANE E. AND IVONA H. MAGOON,P.O. BOX 7070,0200064,DUANE E. AND IVONA H. MAGOONP.O. BOX 7070,2
2,"LARSON, L. LYNN AND SANDRA",213 E. 670 SOUTH,0200096,"LARSON, L. LYNN AND SANDRA213 E. 670 SOUTH",3


In [111]:
# write two columns into a csv that we can import to Django app
authorizations[['auth_no', 'id']].to_csv('data/processed/authorizations.csv')

### Linking authorization numbers to operators 
---

In [112]:
# operators_w_auth_id = pd.merge(operators, authorizations, on='auth_no', how='left')
# operators_w_auth_id.to_csv('data/processed/operators_w_auth_id.csv')
# operators.to_csv('data/processed/operators.csv')

In [113]:
unique_ops_short = unique_ops[['unique', 'operator_id']]
operators_w_id = pd.merge(operators, unique_ops_short, on='unique', how='left')

In [114]:
# check the new dataframe 
operators_w_id.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17863 entries, 0 to 17862
Data columns (total 14 columns):
id                       17863 non-null int64
auth_no                  17863 non-null object
operator_display_name    17863 non-null object
address1                 17863 non-null object
address2                 601 non-null object
city                     17863 non-null object
zipcode15                17859 non-null object
zipcode69                1628 non-null object
ST2                      17863 non-null object
phone_number             4938 non-null object
release_text             17863 non-null object
field_office_id          17863 non-null int64
unique                   17863 non-null object
operator_id              17863 non-null int64
dtypes: int64(3), object(11)
memory usage: 2.0+ MB


In [115]:
authorizations.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17863 entries, 0 to 17862
Data columns (total 5 columns):
operator_display_name    17863 non-null object
address1                 17863 non-null object
auth_no                  17863 non-null object
unique                   17863 non-null object
id                       17863 non-null int64
dtypes: int64(1), object(4)
memory usage: 837.3+ KB


In [116]:
# now merge that new dataframe with authorizations 
auths = authorizations[['auth_no', 'id']]
auths = auths.rename(columns={'id': 'authorization_id'})
operators_w_id_w_auth = pd.merge(operators_w_id, auths, on = 'auth_no')

In [117]:
operators_w_id_w_auth['id'] = operators_w_id_w_auth.index + 1

In [118]:
operators_w_id_w_auth[['id', 'operator_id', 'authorization_id']].to_csv('data/processed/operator_auth_no.csv')

### Allotments table
---

Create a table of operators linked to field offices and operators, along witha unique ID that can be referencecd by other tables such as "health" and "boundary"

In [119]:
#save a copy of all the states to a new csv just because
allotments.to_csv('data/rangeland-administration-system/allotment-info/all_states.csv')

In [120]:
#create a new column with unique id for allotments based on state and allotment number (allotment numbers are unique within states according to BLM documentation)
allotments['allotment_unique'] = allotments['Adm State'] + allotments['Allotment Number']

In [121]:
#check the first few lines to make sure we're all good
allotments["allotment_unique"][:5]
allotments.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25307 entries, 0 to 4190
Data columns (total 16 columns):
Adm State                25307 non-null object
Admin Office             25307 non-null object
Field Office             25307 non-null object
Allotment Number         25307 non-null object
Allotment Name           25307 non-null object
Available For Grazing    25307 non-null object
Grazing Decision         122 non-null object
Public Acres             25215 non-null object
Amp Text                 6869 non-null object
Amp Implement Date       6759 non-null object
Management Stat Text     25307 non-null object
Auth No                  23743 non-null object
Permitted Aums           23629 non-null object
Suspended Aums           18466 non-null object
Susp Use Temp            11974 non-null object
allotment_unique         25307 non-null object
dtypes: object(16)
memory usage: 3.3+ MB


In [122]:
# create a new dataframe and drop the duplicate allotment numbers out of it
allotments_trimmed = allotments[['allotment_unique', 'Admin Office', 'Auth No', 'Allotment Name', 'Allotment Number', 'Available For Grazing', 'Grazing Decision', 'Public Acres', 'Amp Text', 'Amp Implement Date', 'Management Stat Text']]

In [123]:
# drop the duplicates out of it
allotments_trimmed.drop_duplicates('allotment_unique', inplace = True)

In [124]:
# rename columns to get rid of messy spaces and capitals
allotments_trimmed = allotments_trimmed.rename(columns={'Admin Office': 'office_code', 'Auth No': 'auth_no', 'Allotment Number': 'allotment_number', 'Allotment Name': 'allotment_name', 'Available For Grazing': 'available_for_grazing', 'Grazing Decision': 'grazing_decision', 'Public Acres': 'public_acres', 'Amp Text': 'amp_text', 'Amp Implement Date': 'amp_implement_date', 'Management Stat Text': 'management_stat_text'})

In [125]:
# check it out to make sure we're still good ... 
allotments_trimmed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21265 entries, 0 to 4189
Data columns (total 11 columns):
allotment_unique         21265 non-null object
office_code              21265 non-null object
auth_no                  19701 non-null object
allotment_name           21265 non-null object
allotment_number         21265 non-null object
available_for_grazing    21265 non-null object
grazing_decision         122 non-null object
public_acres             21205 non-null object
amp_text                 4820 non-null object
amp_implement_date       4739 non-null object
management_stat_text     21265 non-null object
dtypes: object(11)
memory usage: 1.9+ MB


In [126]:
# join it with field office
allotments_with_field_office = pd.merge(allotments_trimmed, field_offices, on='office_code', how='inner')

In [127]:
#strip the comma out of "public acres" and convert it to a float field so we can calc on it. Then describe() just to make sure it worked
allotments_with_field_office['public_acres'] = allotments_with_field_office['public_acres'].str.replace(',', '')
allotments_with_field_office['public_acres'] = allotments_with_field_office['public_acres'].astype(float)
allotments_with_field_office['public_acres'].describe()

count      21205.000000
mean        7286.288847
std        30588.340811
min            0.000000
25%          240.000000
50%          960.000000
75%         4091.000000
max      1459993.000000
Name: public_acres, dtype: float64

In [128]:
allotments_with_field_office = allotments_with_field_office.rename(columns={'id': 'field_office_id'})

In [129]:
#reassign the variable name to a data frame selecting only the fields we want for our table
allotments_with_field_office = allotments_with_field_office[['allotment_unique', 'allotment_number', 'allotment_name', 'available_for_grazing', 'grazing_decision', 'public_acres', 'amp_text', 'management_stat_text', 'field_office_id', 'amp_implement_date']]

In [130]:
#convert 'amp_implement_date' into a Django-friendly format, then check it with the first five rows jsut to make sure
allotments_with_field_office['amp_implement_date'] = pd.to_datetime(allotments_with_field_office['amp_implement_date'])

# fill the nulls with a common date that none of the actual dates will match and that we can filter out later
# this is to help avoid an error on postgres import
allotments_with_field_office['amp_implement_date'] = allotments_with_field_office['amp_implement_date'].fillna(1930-01-01)

allotments_with_field_office['amp_implement_date'][:10]



0   1981-11-01 00:00:00.000000000
1   1970-01-01 00:00:00.000001928
2   1970-01-01 00:00:00.000001928
3   1983-08-30 00:00:00.000000000
4   1991-06-27 00:00:00.000000000
5   1970-01-01 00:00:00.000001928
6   1971-03-01 00:00:00.000000000
7   1970-01-01 00:00:00.000001928
8   1974-07-01 00:00:00.000000000
9   1976-03-01 00:00:00.000000000
Name: amp_implement_date, dtype: datetime64[ns]

In [131]:
# assign each allotment a unique id
allotments_with_field_office['id'] = allotments_with_field_office.index + 1
allotments_with_field_office['id'][:5]

0    1
1    2
2    3
3    4
4    5
Name: id, dtype: int64

In [132]:
allotments_with_field_office.to_csv('data/processed/allotments.csv')

### Permits table
---

Create a table of permits linked to operators and allotments and with a unique ID that can be referencecd by other tables

In [133]:
permits

,Off CD,Fo Name,Auth No,P/L Eff Dt,P/L exp Dt,Permit Status,Allotment Number,Allotment Name,Lvsk #,Lvsk Kind,Pd Beg Dt,Pd End Dt,Type Use,PL %,Aums
0,LLAZA01000,ARIZONA STRIP FO,0200054,9/1/2007,8/30/2017,NaN,05316,LOST SPRING GAP,12,CATTLE,3/1/2003,4/30/2003,ACTIVE,100,24
1,LLAZA01000,ARIZONA STRIP FO,0200054,9/1/2007,8/30/2017,NaN,05316,LOST SPRING GAP,12,CATTLE,1/1/2003,2/28/2003,ACTIVE,100,23
2,LLAZA01000,ARIZONA STRIP FO,0200064,3/1/2015,2/28/2025,FLPMA 402(C)(2)/APPROP ACT,04810,SULLIVAN CANYON,72,CATTLE,3/1/2003,2/28/2004,ACTIVE,100,864
3,LLAZA01000,ARIZONA STRIP FO,0200096,3/1/2010,2/28/2017,FLPMA 402(C)(2)/APPROP ACT,04842,CEDAR WASH,67,CATTLE,10/16/2004,2/28/2005,ACTIVE,100,300
4,LLAZA01000,ARIZONA STRIP FO,0200096,3/1/2010,2/28/2017,FLPMA 402(C)(2)/APPROP ACT,04842,CEDAR WASH,67,CATTLE,3/1/2005,3/15/2005,ACTIVE,100,33
5,LLAZA01000,ARIZONA STRIP FO,0200097,3/1/2016,2/28/2026,NaN,04856,QUAIL CANYON,68,CATTLE,12/1/2004,2/28/2005,ACTIVE,99,199
6,LLAZA01000,ARIZONA STRIP FO,0200097,3/1/2016,2/28/2026,NaN,04856,QUAIL CANYON,68,CATTLE,3/1/2005,11/30/2005,ACTIVE,99,609
7,LLAZA01000,ARIZONA STRIP FO,0200102,12/1/2014,11/30/2024,FLPMA 402(C)(2)/APPROP ACT,05215,CLAYHOLE,908,CATTLE,12/1/2004,11/30/2005,ACTIVE,86,9371
8,LLAZA01000,ARIZONA STRIP FO,0200105,9/1/2015,8/31/2025,NaN,05210,ANTELOPE SPRING,134,CATTLE,9/16/2005,11/15/2005,ACTIVE,79,212
9,LLAZA01000,ARIZONA STRIP FO,0200105,9/1/2015,8/31/2025,NaN,05210,ANTELOPE SPRING,119,CATTLE,11/16/2005,2/28/2006,ACTIVE,79,325


In [134]:
permits = permits.rename(columns={'Off CD': 'office_code', 'P/L Eff Dt': 'pl_effect_dt', 'P/L exp Dt': 'pl_exp_dt', 'Permit Status': 'permit_status', 'Allotment Number': 'allotment_number', 'Lvsk #': 'livestock_number', 'Lvsk Kind': 'livestock_kind', 'Pd Beg Dt': 'pd_beg_dt', 'Pd End Dt': 'pd_end_dt', 'Type Use': 'type_use', 'PL %': 'pl_percent', 'Aums': 'aums'} )

In [135]:
permits_fo = pd.merge(permits, field_offices, on='office_code', how='inner')

In [136]:
permits_fo = permits_fo.rename(columns={'id': 'field_office_id'})

In [137]:
#convert date fields to datetime
permits_fo['pl_effect_dt'] = pd.to_datetime(permits_fo['pl_effect_dt'])
permits_fo['pl_exp_dt'] = pd.to_datetime(permits_fo['pl_exp_dt'])

permits_fo['pd_beg_dt'] = pd.to_datetime(permits_fo['pd_beg_dt'])
permits_fo['pd_end_dt'] = pd.to_datetime(permits_fo['pd_end_dt'])

In [138]:
permits_fo[:10]

,office_code,Fo Name,Auth No,pl_effect_dt,pl_exp_dt,permit_status,allotment_number,Allotment Name,livestock_number,livestock_kind,pd_beg_dt,pd_end_dt,type_use,pl_percent,aums,field_office_id,office_name,state_id
0,LLAZA01000,ARIZONA STRIP FO,0200054,2007-09-01,2017-08-30,NaN,05316,LOST SPRING GAP,12,CATTLE,2003-03-01,2003-04-30,ACTIVE,100,24,1,ARIZONA STRIP FO,1
1,LLAZA01000,ARIZONA STRIP FO,0200054,2007-09-01,2017-08-30,NaN,05316,LOST SPRING GAP,12,CATTLE,2003-01-01,2003-02-28,ACTIVE,100,23,1,ARIZONA STRIP FO,1
2,LLAZA01000,ARIZONA STRIP FO,0200064,2015-03-01,2025-02-28,FLPMA 402(C)(2)/APPROP ACT,04810,SULLIVAN CANYON,72,CATTLE,2003-03-01,2004-02-28,ACTIVE,100,864,1,ARIZONA STRIP FO,1
3,LLAZA01000,ARIZONA STRIP FO,0200096,2010-03-01,2017-02-28,FLPMA 402(C)(2)/APPROP ACT,04842,CEDAR WASH,67,CATTLE,2004-10-16,2005-02-28,ACTIVE,100,300,1,ARIZONA STRIP FO,1
4,LLAZA01000,ARIZONA STRIP FO,0200096,2010-03-01,2017-02-28,FLPMA 402(C)(2)/APPROP ACT,04842,CEDAR WASH,67,CATTLE,2005-03-01,2005-03-15,ACTIVE,100,33,1,ARIZONA STRIP FO,1
5,LLAZA01000,ARIZONA STRIP FO,0200097,2016-03-01,2026-02-28,NaN,04856,QUAIL CANYON,68,CATTLE,2004-12-01,2005-02-28,ACTIVE,99,199,1,ARIZONA STRIP FO,1
6,LLAZA01000,ARIZONA STRIP FO,0200097,2016-03-01,2026-02-28,NaN,04856,QUAIL CANYON,68,CATTLE,2005-03-01,2005-11-30,ACTIVE,99,609,1,ARIZONA STRIP FO,1
7,LLAZA01000,ARIZONA STRIP FO,0200102,2014-12-01,2024-11-30,FLPMA 402(C)(2)/APPROP ACT,05215,CLAYHOLE,908,CATTLE,2004-12-01,2005-11-30,ACTIVE,86,9371,1,ARIZONA STRIP FO,1
8,LLAZA01000,ARIZONA STRIP FO,0200105,2015-09-01,2025-08-31,NaN,05210,ANTELOPE SPRING,134,CATTLE,2005-09-16,2005-11-15,ACTIVE,79,212,1,ARIZONA STRIP FO,1
9,LLAZA01000,ARIZONA STRIP FO,0200105,2015-09-01,2025-08-31,NaN,05210,ANTELOPE SPRING,119,CATTLE,2005-11-16,2006-02-28,ACTIVE,79,325,1,ARIZONA STRIP FO,1


In [139]:
#strip the two letter state abbr out of the office code
permits_fo['state'] = permits_fo['office_code'].str[2:4]

In [140]:
#create the unique allotment id by concatenating state abbr with allotment number
permits_fo['allotment_unique'] = permits_fo['state'] + permits_fo['allotment_number']

In [141]:
#check to see we got it right 
permits_fo['allotment_unique'][:10]

0    AZ05316
1    AZ05316
2    AZ04810
3    AZ04842
4    AZ04842
5    AZ04856
6    AZ04856
7    AZ05215
8    AZ05210
9    AZ05210
Name: allotment_unique, dtype: object

In [142]:
permits_fo = permits_fo.rename(columns={'Auth No': 'auth_no'})

In [143]:
permits_fo =  permits_fo[['auth_no', 'pl_effect_dt', 'pl_exp_dt', 'permit_status', 'allotment_number', 'livestock_number', 'livestock_kind', 'pd_beg_dt', 'pd_end_dt', 'type_use', 'pl_percent', 'aums', 'field_office_id', 'allotment_unique']]

In [144]:
permits_fo[:10]

,auth_no,pl_effect_dt,pl_exp_dt,permit_status,allotment_number,livestock_number,livestock_kind,pd_beg_dt,pd_end_dt,type_use,pl_percent,aums,field_office_id,allotment_unique
0,0200054,2007-09-01,2017-08-30,NaN,05316,12,CATTLE,2003-03-01,2003-04-30,ACTIVE,100,24,1,AZ05316
1,0200054,2007-09-01,2017-08-30,NaN,05316,12,CATTLE,2003-01-01,2003-02-28,ACTIVE,100,23,1,AZ05316
2,0200064,2015-03-01,2025-02-28,FLPMA 402(C)(2)/APPROP ACT,04810,72,CATTLE,2003-03-01,2004-02-28,ACTIVE,100,864,1,AZ04810
3,0200096,2010-03-01,2017-02-28,FLPMA 402(C)(2)/APPROP ACT,04842,67,CATTLE,2004-10-16,2005-02-28,ACTIVE,100,300,1,AZ04842
4,0200096,2010-03-01,2017-02-28,FLPMA 402(C)(2)/APPROP ACT,04842,67,CATTLE,2005-03-01,2005-03-15,ACTIVE,100,33,1,AZ04842
5,0200097,2016-03-01,2026-02-28,NaN,04856,68,CATTLE,2004-12-01,2005-02-28,ACTIVE,99,199,1,AZ04856
6,0200097,2016-03-01,2026-02-28,NaN,04856,68,CATTLE,2005-03-01,2005-11-30,ACTIVE,99,609,1,AZ04856
7,0200102,2014-12-01,2024-11-30,FLPMA 402(C)(2)/APPROP ACT,05215,908,CATTLE,2004-12-01,2005-11-30,ACTIVE,86,9371,1,AZ05215
8,0200105,2015-09-01,2025-08-31,NaN,05210,134,CATTLE,2005-09-16,2005-11-15,ACTIVE,79,212,1,AZ05210
9,0200105,2015-09-01,2025-08-31,NaN,05210,119,CATTLE,2005-11-16,2006-02-28,ACTIVE,79,325,1,AZ05210


In [145]:
# merge permits with allotments`
permits_fo_allot = pd.merge(permits_fo, allotments_with_field_office, on='allotment_unique', how='inner')

In [146]:
# rename some key columns
permits_fo_allot = permits_fo_allot.rename(columns={'id': 'allotment_id', 'field_office_id_x': 'field_office_id', 'auth_no_x': 'auth_no'})

In [147]:
# select only the columns we need
permits_fo_allot = permits_fo_allot[['auth_no', 'pl_effect_dt', 'pl_exp_dt', 'permit_status', 'livestock_number', 'livestock_kind', 'pd_beg_dt', 'pd_end_dt', 'type_use', 'pl_percent', 'aums', 'field_office_id', 'allotment_unique', 'allotment_id']]

In [148]:
permits_fo_allot_auths = pd.merge(permits_fo_allot, auths, on='auth_no', how='inner')

In [149]:
permits_fo_allot_auths = permits_fo_allot_auths.rename(columns={'authorization_id': 'auth_no_id', 'field_office_id_x': 'field_office_id'})

In [150]:
permits_fo_allot_auths = permits_fo_allot_auths[['auth_no', 'pl_effect_dt', 'pl_exp_dt', 'permit_status', 'livestock_number', 'livestock_kind', 'pd_beg_dt', 'pd_end_dt', 'type_use', 'pl_percent', 'aums', 'field_office_id', 'allotment_unique', 'allotment_id', 'auth_no_id']]

In [151]:
permits_fo_allot_auths['id'] = permits_fo_allot_auths.index + 1
permits_fo_allot_auths[:2]

,auth_no,pl_effect_dt,pl_exp_dt,permit_status,livestock_number,livestock_kind,pd_beg_dt,pd_end_dt,type_use,pl_percent,aums,field_office_id,allotment_unique,allotment_id,auth_no_id,id
0,0200054,2007-09-01,2017-08-30,NaN,12,CATTLE,2003-03-01,2003-04-30,ACTIVE,100,24,1,AZ05316,99,1,1
1,0200054,2007-09-01,2017-08-30,NaN,12,CATTLE,2003-01-01,2003-02-28,ACTIVE,100,23,1,AZ05316,99,1,2


In [152]:
permits_fo_allot_auths['pl_effect_dt'] = permits_fo_allot_auths['pl_effect_dt'].fillna(1930-01-01)
permits_fo_allot_auths['pl_exp_dt'] = permits_fo_allot_auths['pl_exp_dt'].fillna(1930-01-01)
permits_fo_allot_auths['pd_beg_dt'] = permits_fo_allot_auths['pd_beg_dt'].fillna(1930-01-01)
permits_fo_allot_auths['pd_end_dt'] = permits_fo_allot_auths['pd_end_dt'].fillna(1930-01-01)

In [153]:
permits_fo_allot_auths.to_csv('data/processed/permits.csv')

### Allotments and authorizations many-to-many table
----
An outer join of allotment ids, numbers and operator ids, numbers for the Many to Many relationship between allotments and operators

In [154]:
allotments = allotments.rename(columns={'Auth No': 'auth_no'})
allotments_operators = pd.merge(auths, allotments, on="auth_no", how='left')
#allotments_operators = allotments_operators.rename(columns={'id': 'operator_id'})

In [155]:
allotments_operators = pd.merge(allotments_operators, allotments_with_field_office, on='allotment_unique')

In [156]:
allotments_operators = allotments_operators.rename(columns={'id': 'allotment_id'})

In [157]:
allotments_operators = allotments_operators[['allotment_id', 'authorization_id']]

In [158]:
allotments_operators['id'] = allotments_operators.index + 1

In [159]:
allotments_operators.to_csv('data/processed/allotments_auth_no.csv')

### Health table
---
Convert column names, match up with allotments table

In [237]:
#Rename column headers, breaking it up because column names are very long and I kept making syntax errors
health = health.rename(columns={'Admin State': 'admin_state', 'Allotment Name': 'allotment_name', 'Date NEPA Analysis Completed (mm/dd/yyyy)5': 'nepa_date'})
health = health.rename(columns={'Date of Most Recent Land Health Evaluation Report (mm/dd/yyyy)1': 'land_health_eval_date'})
health = health.rename(columns={'Date of most recent Determination of Causal Factor(s) (mm/dd/yyyy)2': 'causal_factors_date'})
health = health.rename(columns={'Land Health Standard(s) Not Achieved in the Allotment and Significant Causal Factor(s) Identified3': 'description'})
health = health.rename(columns={'Allotment Number': 'allotment_number', 'Authorization Number': 'auth_no', 'Office Code': 'office_code', 'Permit or Lease Status': 'permit_status', 'Type of NEPA Analysis for Grazing Authorization (EA, EIS, CX, DNA)4': 'nepa_type'})

In [238]:
health['allotment_unique'] = health['admin_state'] + health['allotment_number']

In [239]:
health['allotment_unique'][:5]

0    AZ05206
1    AZ05210
2    AZ05207
3    AZ05334
4    AZ04828
Name: allotment_unique, dtype: object

In [240]:
health_w_allot = pd.merge(health, allotments_with_field_office, on='allotment_unique', how='left')

In [242]:
health_w_allot.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23826 entries, 0 to 23825
Data columns (total 36 columns):
admin_state              23825 non-null object
allotment_name_x         23824 non-null object
allotment_number_x       23825 non-null object
auth_no                  23803 non-null object
nepa_date                20254 non-null object
land_health_eval_date    22598 non-null object
causal_factors_date      19435 non-null object
description              18147 non-null object
NEPA Identifier6         20273 non-null object
office_code              23825 non-null object
permit_status            23124 non-null object
nepa_type                21700 non-null object
Unnamed: 12              0 non-null float64
Unnamed: 13              0 non-null float64
Unnamed: 14              0 non-null float64
Unnamed: 15              0 non-null float64
Unnamed: 16              0 non-null float64
Unnamed: 17              0 non-null float64
Unnamed: 18              0 non-null float64
Unnamed: 19        

In [243]:
health_w_allot = health_w_allot[['admin_state', 'allotment_name_x', 'allotment_number_x', 'auth_no', 'nepa_date', 'land_health_eval_date', 'causal_factors_date', 'description', 'NEPA Identifier6', 'office_code', 'permit_status', 'nepa_type', 'allotment_unique', 'allotment_number_y', 'allotment_name_y', 'available_for_grazing', 'grazing_decision', 'public_acres', 'amp_text', 'management_stat_text', 'field_office_id', 'amp_implement_date', 'id']]

In [244]:
health_w_allot = health_w_allot.rename(columns={'id': 'allotment_id'})

In [246]:
health_w_allot.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23826 entries, 0 to 23825
Data columns (total 23 columns):
admin_state              23825 non-null object
allotment_name_x         23824 non-null object
allotment_number_x       23825 non-null object
auth_no                  23803 non-null object
nepa_date                20254 non-null object
land_health_eval_date    22598 non-null object
causal_factors_date      19435 non-null object
description              18147 non-null object
NEPA Identifier6         20273 non-null object
office_code              23825 non-null object
permit_status            23124 non-null object
nepa_type                21700 non-null object
allotment_unique         23825 non-null object
allotment_number_y       23716 non-null object
allotment_name_y         23716 non-null object
available_for_grazing    23716 non-null object
grazing_decision         14 non-null object
public_acres             23654 non-null float64
amp_text                 6764 non-null object
m

In [247]:
# convert to acceptable datetimes.
health_w_allot['nepa_date'] = pd.to_datetime(health_w_allot['nepa_date'])
health_w_allot['land_health_eval_date'] = pd.to_datetime(health_w_allot['land_health_eval_date'])
health_w_allot['causal_factors_date'] = pd.to_datetime(health_w_allot['causal_factors_date'])

# if null, fill with this easily filtered out date
health_w_allot['nepa_date'] = health_w_allot['nepa_date'].fillna(1930-01-01)
health_w_allot['land_health_eval_date'] = health_w_allot['land_health_eval_date'].fillna(1930-01-01)
health_w_allot['causal_factors_date'] = health_w_allot['causal_factors_date'].fillna(1930-01-01)

In [253]:
health_w_allot.groupby('description').count()

,admin_state,allotment_name_x,allotment_number_x,auth_no,nepa_date,land_health_eval_date,causal_factors_date,NEPA Identifier6,office_code,permit_status,...,allotment_number_y,allotment_name_y,available_for_grazing,grazing_decision,public_acres,amp_text,management_stat_text,field_office_id,amp_implement_date,allotment_id
description,,,,,,,,,,,,,,,,,,,,,
,9,9,9,9,9,9,9,9,9,9,...,9,9,9,0,9,8,9,9,9,9
Oil and Gas Operations,1,1,1,1,1,1,1,1,1,1,...,1,1,1,0,1,0,1,1,1,1
4 - Habitat Supports Species Diversity Periodic Drought,1,1,1,1,1,1,1,1,1,1,...,1,1,1,0,1,1,1,1,1,1
3 - Upland Vegetation Condition Current Livestock Grazing,1,1,1,1,1,1,1,1,1,1,...,1,1,1,0,1,0,1,1,1,1
3 - Upland Vegetation Condition INPS,4,4,4,4,4,4,4,4,4,4,...,4,4,4,0,4,0,4,4,4,4
3 - Upland Vegetation Condition INPS 4 - Habitat Supports Species Diversity INPS,1,1,1,1,1,1,1,1,1,1,...,1,1,1,0,1,0,1,1,1,1
3 - Upland Vegetation Condition Unknown,1,1,1,1,1,1,1,0,1,1,...,1,1,1,0,1,0,1,1,1,1
2 - Riparian Condition Current Livestock Grazing,5,5,5,5,5,5,5,5,5,5,...,5,5,5,0,5,0,5,5,5,5
2 - Riparian Condition Current Livestock Grazing 3 - Upland Vegetation Condition Current Livestock Grazing,6,6,6,6,6,6,6,6,6,6,...,6,6,6,0,6,1,6,6,6,6


In [219]:
health_w_allot.loc[health_w_allot['description'].str.contains("All standards are met", na=False), 'land_health_status'] = 'Y'
health_w_allot

# Check whether livestock is considered a factor
        # health_w_allot.loc[health_w_allot['description'].str.contains("Livestock", na=False), 'livestock_factor'] = 'Yes'
        # health_w_allot.loc[health_w_allot['description'].str.contains("livestock", na=False), 'livestock_factor'] = 'Yes'
        # health_w_allot['livestock_factor'] = health_w_allot['livestock_factor'].fillna('No')

In [225]:
health_uniques = pd.DataFrame()

In [227]:
health_uniques = health_uniques.append(health)

In [230]:
health_uniques['allotment_unique'] = health_uniques['admin_state'] + health_uniques['allotment_number']

In [232]:
health_uniques.drop_duplicates('allotment_unique', inplace=True)

In [235]:
health_uniques.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19753 entries, 0 to 4112
Data columns (total 26 columns):
admin_state              19752 non-null object
allotment_name           19751 non-null object
allotment_number         19752 non-null object
auth_no                  19730 non-null object
nepa_date                16960 non-null object
land_health_eval_date    18726 non-null object
causal_factors_date      15852 non-null object
description              15069 non-null object
NEPA Identifier6         16996 non-null object
office_code              19752 non-null object
permit_status            19093 non-null object
nepa_type                18116 non-null object
Unnamed: 12              0 non-null float64
Unnamed: 13              0 non-null float64
Unnamed: 14              0 non-null float64
Unnamed: 15              0 non-null float64
Unnamed: 16              0 non-null float64
Unnamed: 17              0 non-null float64
Unnamed: 18              0 non-null float64
Unnamed: 19         

# Analysis for story

The number and percent of allotments lacking allotment managemnet plans:

The number and percent of existing allotment management plans implemented since 2000:

The number and percent of permits renewed under the appropriations act, without an environmental analysis:

The number and percent of permits renewed under the appropriations act (without an environmental analysis) that are in the "improve" category.

The number and percent of permits renewed under the appropriations act (without an environmental analysis) that have an an environmental asessemnt and did not meet land health standards (livestock vs not livestock).